In [1]:
# dependencies
import json
import requests
import pandas as pd
from pprint import pprint
# need to pip install world bank data to access more detailed info
# pip install -U wbdata
import wbdata
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy
import math


#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

In [19]:
# owid json on GitHub url
owid_covid19_url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.json"

# Media Group API urls
cases="https://covid-api.mmediagroup.fr/v1/history?country=all&status=confirmed"
deaths="https://covid-api.mmediagroup.fr/v1/history?country=all&status=deaths"

In [20]:
# owid json json requests
owid_vaccinations_response = requests.get(owid_covid19_url).json()


# Media Group json requests
case_response=requests.get(cases).json()
# temp need to be changed to case_response

deaths_response=requests.get(deaths).json()
# temp2 need to be changed to deaths_response


In [21]:
### owid json on GitHub of vaccinations - API call and DataFrame creation

json_dict = {
    "country": [],
    "iso_code": [],
    "date": [],
    "daily_vaccinations": [],
    "daily_vaccinations_per_million": [],
    "people_fully_vaccinated_per_hundred" : [],
    "people_vaccinated_per_hundred" : []
}

for index, x in enumerate(owid_vaccinations_response):
    results = owid_vaccinations_response[index]

    for index, data in enumerate(results["data"]):
        json_dict["country"].append(results["country"])
        json_dict["iso_code"].append(results["iso_code"])
            
        json_dict["date"].append(results["data"][index]["date"])
            
        try:
            json_dict["daily_vaccinations"].append(results["data"][index]["daily_vaccinations"])
        except(KeyError):
            json_dict["daily_vaccinations"].append("No Data")
                
        try:    
            json_dict["daily_vaccinations_per_million"].append(results["data"][index]["daily_vaccinations_per_million"])
        except(KeyError):
            json_dict["daily_vaccinations_per_million"].append("No Data")
        
        try:    
            json_dict["people_fully_vaccinated_per_hundred"].append(results["data"][index]["people_fully_vaccinated_per_hundred"])
        except(KeyError):
            json_dict["people_fully_vaccinated_per_hundred"].append(0)
            
        try:    
            json_dict["people_vaccinated_per_hundred"].append(results["data"][index]["people_vaccinated_per_hundred"])
        except(KeyError):
            json_dict["people_vaccinated_per_hundred"].append(None)
            

raw_owid_vaccinations_data = pd.DataFrame(json_dict)

april_owid_vaccinations_data = raw_owid_vaccinations_data.loc[raw_owid_vaccinations_data["date"].str.contains("2021-04"), :]
april_owid_vaccinations_data.head()

,country,iso_code,date,daily_vaccinations,daily_vaccinations_per_million,people_fully_vaccinated_per_hundred,people_vaccinated_per_hundred
38,Afghanistan,AFG,2021-04-01,3000,77,0.0,NaN
39,Afghanistan,AFG,2021-04-02,3000,77,0.0,NaN
40,Afghanistan,AFG,2021-04-03,3000,77,0.0,NaN
41,Afghanistan,AFG,2021-04-04,3000,77,0.0,NaN
42,Afghanistan,AFG,2021-04-05,3000,77,0.0,NaN


In [22]:
the_confirmed_dates=[]
the_confirmed=[]
the_country=[]

# temp need to be changed to case_response

#finds the dates and confirmed cases for all the countries
for i in case_response:
    the_confirmed_temp=case_response[i]['All']['dates']
    for d in the_confirmed_temp:
        the_country.append(i)
        the_confirmed_dates.append(d)
        the_confirmed.append(the_confirmed_temp[d])

the_death_dates=[]
the_death=[]
the_country=[]


# temp2 need to be changed to deaths_response

for i in deaths_response:
    the_death_temp=deaths_response[i]['All']['dates']
    for d in the_death_temp:
        the_country.append(i)
        the_death_dates.append(d)
        the_death.append(the_death_temp[d])
        
confirmed_dict={'Location': the_country,'Date':the_confirmed_dates,"Total_Confirmed":the_confirmed,"Total_Deaths":the_death}   
confirmed_df=pd.DataFrame(confirmed_dict)

the_list=[confirmed_df['Location'],confirmed_df['Date'],confirmed_df['Total_Confirmed'],confirmed_df['Total_Deaths']]

daily_confirmed=[]
daily_deaths=[]

for row in range(0,len(the_list[0]),1):
    if row != len(the_list[0])-1:
        if the_list[0][row]==the_list[0][row+1]: #same country
            daily_confirmed.append(the_list[2][row] - the_list[2][row+1])
            daily_deaths.append(the_list[3][row] - the_list[3][row+1])
        else: #at a new country
            daily_confirmed.append(the_list[2][row])
            daily_deaths.append(the_list[3][row])
    else:
        daily_confirmed.append(the_list[2][row])
        daily_deaths.append(the_list[3][row])
        

#outputs all the dates for all the countries
#print()
#print()
#print("         Covid Cases All Countries: 01/22/2020 to 04/30/2021")
#print("---------------------------------------------------------------------")
confirmed_df["Daily_Confirmed"]=daily_confirmed
confirmed_df["Daily_Deaths"]=daily_deaths
#confirmed_df


#outputs just april of 2021
print()
print()
print("         Covid Cases All Countries: April 2021)")
print("---------------------------------------------------------------------")
      
april_mediagroup_df=confirmed_df.loc[confirmed_df['Date'].str.contains("2021-04")]
#april_mediagroup_df.to_csv("output/april_covid_rates.csv", index=False)
april_mediagroup_df.head()



         Covid Cases All Countries: April 2021)
---------------------------------------------------------------------


,Location,Date,Total_Confirmed,Total_Deaths,Daily_Confirmed,Daily_Deaths
6,Afghanistan,2021-04-30,59745,2625,169,7
7,Afghanistan,2021-04-29,59576,2618,206,7
8,Afghanistan,2021-04-28,59370,2611,145,13
9,Afghanistan,2021-04-27,59225,2598,210,6
10,Afghanistan,2021-04-26,59015,2592,172,10


In [23]:
# testing wbdata install
#wbdata.get_source()
# finding available indicators to refine data
#wbdata.search_indicators('gdp per capita')
# confirming country list
#wbdata.get_country()


# iterating through countries to get their Gross Domestic Product per Capita
countries = [i['name'] for i in wbdata.get_country()]

indicators = { "NY.GDP.PCAP.CD" : "gdppc"}

# Sorting new data into a dataframe
wbdata_df = wbdata.get_dataframe(indicators, country='All')
wbdata_df = wbdata_df.reset_index()

wbdata_2019_df = wbdata_df[wbdata_df.date == "2019"]

#import pdb; pdb.set_trace()

clean_wbdata_2019_df = wbdata_2019_df


# get rid of null columns
#clean_wbdata_2019_df = clean_wbdata_2019_df.dropna()
clean_wbdata_2019_df = clean_wbdata_2019_df.reset_index(drop=True)

# Verifying dataframe is how we want it
format_dict = {'gdppc':'${0:,.2f}'}
clean_wbdata_2019_df.style.format(format_dict).hide_index()
clean_wbdata_2019_df.head()

,country,date,gdppc
0,Arab World,2019,6584.740241
1,Caribbean small states,2019,10500.971902
2,Central Europe and the Baltics,2019,16298.478764
3,Early-demographic dividend,2019,3644.014158
4,East Asia & Pacific,2019,11502.865111


In [24]:
#this used to pull from csv files before notebooks were merged, now it pulls from original API pull (kept original variable names though)
covid_csv_2021=confirmed_df#this was merged from another notebook, so changhing variable name to that it still works
vaccines_csv=raw_owid_vaccinations_data#this was merged from another notebook, so changhing variable name to that it still works

In [25]:
#cleans country names to match
vaccines_csv['country']=vaccines_csv['country'].replace({'United States': 'US'})
covid_csv_2021['Location']=covid_csv_2021['Location'].replace({'Congo (Brazzaville)': 'Congo', 'Congo (Kinshasa)': 'Congo'})
                                                             

In [26]:
#creates a new df with reduced columns for vaccines
vaccines_df=vaccines_csv[["country",'date','daily_vaccinations','daily_vaccinations_per_million','people_vaccinated_per_hundred','people_fully_vaccinated_per_hundred']]

#renames matching columns
vaccine_df_renamed=vaccines_df.rename(columns={"date":"Date","country":"Location"})

#just grabs apil dates
#apr_vaccines_df=vaccine_df_renamed.loc[vaccine_df_renamed['Date'].str.contains("2021-04")]
#apr_vaccines_df.head()

In [27]:
#merges on two columns going back to January 2021
merged_df_2021=pd.merge(covid_csv_2021,vaccine_df_renamed, on=["Date","Location"], how="inner")

In [38]:
merged_df_2021.head()

,Location,Date,Total_Confirmed,Total_Deaths,Daily_Confirmed,Daily_Deaths,daily_vaccinations,daily_vaccinations_per_million,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred
0,Afghanistan,2021-04-22,58312,2561,98,4,8000,206,0.62,0.0
1,Afghanistan,2021-04-21,58214,2557,177,8,8000,206,NaN,0.0
2,Afghanistan,2021-04-20,58037,2549,139,3,8000,206,NaN,0.0
3,Afghanistan,2021-04-19,57898,2546,105,7,8000,206,NaN,0.0
4,Afghanistan,2021-04-18,57793,2539,72,0,8000,206,NaN,0.0


In [29]:
#merged_df_2021.to_csv("output/2021_covid_vacc_merged.csv", index=False)

In [39]:
#cleans of data frames for merging

clean_wbdata_2019_df=clean_wbdata_2019_df.rename(columns={'country':'Location'})
clean_wbdata_2019_df['Location']=clean_wbdata_2019_df['Location'].replace({'United States': 'US',
                                                                           'Congo, Dem. Rep.':'Congo',
                                                                          'Congo, Rep':'Congo'})

clean_wbdata_2019_df=clean_wbdata_2019_df[['Location','gdppc']]

In [40]:
merged_2021_GDP=pd.merge(merged_df_2021, clean_wbdata_2019_df, on=["Location"], how="left")

In [42]:
merged_2021_GDP['gdppc']=round(merged_2021_GDP['gdppc'],2)
merged_2021_GDP_rename=merged_2021_GDP.rename(columns={'gdppc':'GDPPC (2019)'})
merged_2021_GDP_rename.head()

,Location,Date,Total_Confirmed,Total_Deaths,Daily_Confirmed,Daily_Deaths,daily_vaccinations,daily_vaccinations_per_million,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,GDPPC (2019)
0,Afghanistan,2021-04-22,58312,2561,98,4,8000,206,0.62,0.0,507.1
1,Afghanistan,2021-04-21,58214,2557,177,8,8000,206,NaN,0.0,507.1
2,Afghanistan,2021-04-20,58037,2549,139,3,8000,206,NaN,0.0,507.1
3,Afghanistan,2021-04-19,57898,2546,105,7,8000,206,NaN,0.0,507.1
4,Afghanistan,2021-04-18,57793,2539,72,0,8000,206,NaN,0.0,507.1


In [47]:
#outputs data to csv
#merged_2021_GDP.to_csv('output_csv/covid_vaccine_gdp_merged.csv')